In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


0% [Working]
            
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease

0% [Connecting to archive.ubuntu.com] [Connecting to security.ubuntu.com (185.1
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease

0% [Connecting to archive.ubuntu.com (185.125.190.36)] [Waiting for headers] [W
                                                                               
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.lau

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales.show()


In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView('home_sale')


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg3_df = home_sales.filter(home_sales["bedrooms"]==4)
avg3_df.groupBy("date_built").agg({"price":"avg"}).show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg4_df = home_sales.filter((home_sales["bedrooms"]==3) & (home_sales["bathrooms"]==3))
avg4_df.groupBy("date_built").agg({"price":"avg"}).show()

In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg5_df = home_sales.filter((home_sales["bedrooms"]==3) & (home_sales["bathrooms"]==3) & (home_sales["floors"]==2) & (home_sales["sqft_living"]>1999))
avg5_df.groupBy("date_built").agg({"price":"avg"}).show()


In [1]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT 
  ' price', 
  'view',
  count(*) as count
FROM home_sale
WHERE ' price' > 349999 
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sale")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sale')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT 
  ' price', 
  'view',
  count(*) as count
FROM home_sale
WHERE ' price' > 349999 
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales.write.partitionBy("date_built").mode("overwrite").parquet('parquet_home_sales2')

In [ ]:
# 11. Read the parquet formatted data.
home_sales2=spark.read.parquet('parquet_home_sales2')

In [ ]:
# 12. Create a temporary table for the parquet data.
home_sales2.createOrReplaceTempView('home_sales2')

In [2]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT 
  ' price', 
  'view',
  count(*) as count
FROM home_sale
WHERE ' price' > 349999 
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sale")

In [ ]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached("home_sale"):
    print("a table is till cached")
else:
    print("all clear")